In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node
rm -rf /shared/CSHL_data_processed/MD590/MD590_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD590
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD590/MD590_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropbox.txt /shared/CSHL_data_processed/MD590/MD590_alignedTo_MD590-N47-2015.09.12-05.32.06_MD590_2_0140_cropbox.txt


No vtk
Child returned 0
Child returned 0
0.38 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD591/MD591_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD591
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD591/MD591_sorted_filenames.txt /shared/CSHL_data_processed/MD591/MD591_sorted_filenames.txt


Child returned 0
0.44 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD591/MD591_anchor.txt && mkdir -p /shared/CSHL_data_processed/MD591
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD591/MD591_anchor.txt /shared/CSHL_data_processed/MD591/MD591_anchor.txt


...

Child returned 0
0.42 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD658/MD658_anchor.txt && mkdir -p /shared/CSHL_data_processed/MD658
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD658/MD658_anchor.txt /shared/CSHL_data_processed/MD658/MD658_anchor.txt


Child returned 0
0.42 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD658
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropbox.txt /shared/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropbox.txt


Child returned 0
0.40 seconds.


# Compute scoremaps for linearly normalized version

In [18]:
stack = 'MD657'

for classifier_id in range(83, 85):
    t = time.time()
    sys.stderr.write('running svm classifier ...')

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v4.py'),
                    'stack': stack,
                    'classifier_id': classifier_id},
    #                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                    kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                                               if fn.split('-')[1][0] == 'F']),
                    argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

    t = time.time()
    sys.stderr.write('Resampling scoremaps ...')

    # downscale = 8
    downscale = 32

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v4.py'),
                    'stack': stack,
                    'classifier_id': classifier_id,
                    'downscale': downscale},
    #                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                    kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                               if fn.split('-')[1][0] == 'F']),
                    argument_type='list2'
                   )


    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 317.484612 seconds
Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 70.839120 seconds
running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 322.478262 seconds
Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 65.922713 seconds


# Regular version

In [13]:
classifier_id = 37

In [14]:
stack = 'MD657'

In [15]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id},
#                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                                           if fn.split('-')[1][0] == 'F']),
                argument_type='list2')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 317.269897 seconds


In [16]:
# Feature computation window spacing is 56, so the equivalent downscale factor is 56. Still larger than 32.

In [17]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

# downscale = 8
downscale = 32

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': downscale},
#                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                           if fn.split('-')[1][0] == 'F']),
                argument_type='list2'
               )


wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 60.760376 seconds


In [12]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
# viz_downscale = 8
viz_downscale = 32

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v4.py'),
                'stack': stack,
                 'classifier_id': classifier_id,
                'downscale': viz_downscale,
                'add_label_text': '-a' if add_label_text else ''},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                argument_type='list2'
               )

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # For one stack, 75s

visualize scoremaps ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 20.180460 seconds


In [4]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

use_nissl_only = False
volume_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(classifier_id)d %(downscale)d %(use_nissl_only)s' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': volume_downscale,
                'use_nissl_only': '-n' if use_nissl_only else ''},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 116s

constructing score volumes ...Child returned 0
14 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 146.254704 seconds


In [5]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(classifier_id)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v4.py'),
                'stack': stack,
                'classifier_id': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 55 seconds (16 nodes write to respective local /scratch)
# More than 1 simul. processes are not beneficial as they cause local write contention.
# 156 seconds (16 nodes simul. write to /shared)
# 310 seconds (single node, sequential write to /shared)

compute score volume gradients...Child returned 0
14 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 55.756982 seconds


# Global alignment

In [2]:
classifier_setting = 37
warp_setting = 1

In [21]:
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 'VLL_L', 'VLL_R', '7n_L']
structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 
                    'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
                    'VCA_L', 'VCP_R']
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 
#                     'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
#                     'VCA_L', 'VCP_R', 'IC', 'SC']

In [27]:
atlas_name = 'atlasV3'

In [26]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d --trial_num 1 --structures \'%(structures)s\'' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v3.py'),
                 'atlas_name': atlas_name,
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting,
                'structures': json.dumps(structure_subset)},
                 kwargs_list=dict(stack=['MD635']),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 372s (1 trial); 750s (5 trials)

align all subjects to atlas ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 261.050432 seconds


In [28]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v4.py'),
                'atlas_name': atlas_name,
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 60 seconds

transform atlas ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 40.204390 seconds


In [29]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v4.py'),
                'atlas_name': atlas_name,
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 110 seconds

visualize aligned atlas overlay ...Child returned 0
1 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 90.381931 seconds


# Local align

In [11]:
classifier_setting = 37
warp_setting = 8

In [ ]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

kwargs_list = \
[dict(stack='MD635', structures=shell_escape(json.dumps(all_known_structures_sided[f:l+1])))
for f,l in first_last_tuples_distribute_over(0, len(all_known_structures_sided)-1, get_num_nodes())]

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d -s %%(structures)s' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'local_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=kwargs_list,
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2047s for 5 trials.
# IC and SC are super slow...

In [13]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v4.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 60 seconds

transform atlas ...Child returned 0
14 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 50.400943 seconds


In [14]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed(command='%(script_path)s %%(stack)s atlasV3 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v4.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': ['MD635']},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 110 seconds

visualize aligned atlas overlay ...Child returned 0
14 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 95.670595 seconds


# Transform locally transformed volumes back to atlas space

In [ ]:
classifier_setting = 2
warp_setting = 3

In [ ]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD603']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

# Measure Global Tx Confidence

In [ ]:
warp_setting = 1

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

# Measure Local Tx Confidence

In [ ]:
warp_setting = 5
classifier_setting = 2

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 s ~ 15 mins